<a href="https://colab.research.google.com/github/kesanir/ML-AI-TRADING/blob/main/Copy_of_Momentumscan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================
# 1️⃣ Install Required Packages
# ==============================
!pip install yfinance plotly hmmlearn torch scikit-learn --quiet

import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from hmmlearn.hmm import GaussianHMM
import torch
from datetime import datetime
from IPython.display import display, HTML

# ==============================
# 2️⃣ Parameters
# ==============================
TICKERS = [
    "AAPL","MSFT","NVDA","AMZN","META","TSLA","AVGO","AMD","GOOGL",
    "NFLX","COST","ADBE","PANW","SMCI","CRWD","ASML","MRVL","MU",
    "S","FTNT","ZS"
]

SEQ_LEN = 20

# ==============================
# 3️⃣ Market Regime (HMM on QQQ)
# ==============================
def get_market_regime():
    qqq = yf.download("QQQ", period="3y", progress=False, auto_adjust=True)
    returns = np.log(qqq['Close']).diff().dropna().values.reshape(-1,1)
    model = GaussianHMM(n_components=3, n_iter=500)
    model.fit(returns)
    state = model.predict(returns)[-1]
    labels = {0:"BULL",1:"NEUTRAL",2:"BEAR"}
    return labels[state], qqq

regime, qqq = get_market_regime()

display(HTML(f"<h2 style='color:green'>Market Regime: {regime}</h2>"))

# ==============================
# 4️⃣ Indicator Calculations
# ==============================
def compute_indicators(df):
    df = df.copy()
    df['RSI'] = 100 - (100 / (1 + df['Close'].diff().clip(lower=0).rolling(14).mean() /
                              (-df['Close'].diff().clip(upper=0)).rolling(14).mean()))
    df['SMA20'] = df['Close'].rolling(20).mean()
    df['SMA50'] = df['Close'].rolling(50).mean()
    df['ROC10'] = df['Close'].pct_change(10)
    df['VOLR'] = df['Volume'] / df['Volume'].rolling(20).mean()
    return df.dropna()

# ==============================
# 5️⃣ Generate Momentum Signals
# ==============================
def build_signals():
    rows = []
    for t in TICKERS:
        df = yf.download(t, period="1y", progress=False, auto_adjust=True)
        df = compute_indicators(df)
        if len(df) < 60:
            continue

        # Get scalar values from the last row using .item() to ensure pure scalars
        last_close = df["Close"].iloc[-1].item()
        last_sma20 = df["SMA20"].iloc[-1].item()
        last_sma50 = df["SMA50"].iloc[-1].item()
        last_rsi = df["RSI"].iloc[-1].item()
        last_roc10 = df["ROC10"].iloc[-1].item()
        last_volr = df["VOLR"].iloc[-1].item()

        momentum_score = (
            int(last_close > last_sma20) * 20 +
            int(last_sma20 > last_sma50) * 20 +
            int(last_rsi > 55) * 20 +
            int(last_roc10 > 0) * 20 +
            int(last_volr > 1.3) * 20
        )
        ai_prob = min(0.95, momentum_score / 100 + np.random.normal(0, 0.05))
        rows.append({
            "Ticker": t,
            "Price": round(last_close,2),
            "Momentum_Score": momentum_score,
            "Momentum_Probability": round(ai_prob,2),
            "Volume_Ratio": round(last_volr,2)
        })
    return pd.DataFrame(rows).sort_values("Momentum_Probability", ascending=False)

signals = build_signals()
display(HTML("<h3>Top Momentum Signals</h3>"))
display(signals.head(15))

# ==============================
# 6️⃣ Signal Confidence Distribution
# ==============================
fig_conf = px.histogram(signals, x="Momentum_Probability", nbins=20,
                        title="Signal Confidence Distribution")
fig_conf.show()

# ==============================
# 7️⃣ Correlation Heatmap
# ==============================
prices = yf.download(TICKERS, period="6mo", auto_adjust=True)['Close']
returns = prices.pct_change().dropna()
corr = returns.corr()
fig_corr = px.imshow(corr, color_continuous_scale="RdBu", title="Correlation Heatmap")
fig_corr.show()

# ==============================
# 8️⃣ Simulated Equity Curve
# ==============================
equity = (1 + signals["Momentum_Probability"].values * 0.01).cumprod()
# Ensure equity is a 1D numpy array before passing to plotly.express
fig_eq = px.line(y=equity.flatten(), title="Simulated Equity Curve (Model Weighted)")
fig_eq.show()

# ==============================
# 9️⃣ Stock Drill-Down (Interactive)
# ==============================
import ipywidgets as widgets
from IPython.display import display

ticker_dropdown = widgets.Dropdown(options=signals["Ticker"], description='Ticker:')
output = widgets.Output()

def plot_ticker(change):
    output.clear_output()
    t = change['new']
    df = yf.download(t, period="6mo", auto_adjust=True)
    fig_price = px.line(df, y="Close", title=f"{t} Price (6 Months)")
    with output:
        fig_price.show()

ticker_dropdown.observe(plot_ticker, names='value')
display(ticker_dropdown, output)


,Ticker,Price,Momentum_Score,Momentum_Probability,Volume_Ratio
15,ASML,1450.56,80,0.86,0.87
10,COST,1010.79,80,0.83,1.77
9,NFLX,96.24,80,0.83,3.70
16,MRVL,81.69,60,0.61,1.53
0,AAPL,264.18,60,0.60,1.34
17,MU,412.37,60,0.60,0.79
13,SMCI,32.39,60,0.60,0.80
2,NVDA,177.19,40,0.42,1.58
4,META,648.18,20,0.25,1.12
19,FTNT,79.03,20,0.19,0.94


[*********************100%***********************]  21 of 21 completed


Dropdown(description='Ticker:', options=('ASML', 'COST', 'NFLX', 'MRVL', 'AAPL', 'MU', 'SMCI', 'NVDA', 'META',…

Output()

In [ ]:
# ==============================
# 1️⃣ Install Required Packages
# ==============================
!pip install yfinance plotly torch scikit-learn hmmlearn --quiet

import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from hmmlearn.hmm import GaussianHMM
import plotly.express as px
from datetime import datetime
from IPython.display import display, HTML

# ==============================
# 2️⃣ Parameters
# ==============================
TICKERS = [
    "AAPL","MSFT","NVDA","AMZN","META","TSLA","AVGO","AMD","GOOGL",
    "NFLX","COST","ADBE","PANW","SMCI","CRWD","ASML","MRVL","MU"
]
SEQ_LEN = 20
FUTURE_DAYS = 5
THRESHOLD = 0.02  # 2% price move

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

# ==============================
# 3️⃣ Market Regime (HMM on QQQ)
# ==============================
def get_market_regime():
    qqq = yf.download("QQQ", period="3y", progress=False, auto_adjust=True)
    returns = np.log(qqq['Close']).diff().dropna().values.reshape(-1,1)
    model = GaussianHMM(n_components=3, n_iter=500)
    model.fit(returns)
    state = model.predict(returns)[-1]
    labels = {0:"BULL",1:"NEUTRAL",2:"BEAR"}
    return labels[state], qqq

regime, qqq = get_market_regime()
display(HTML(f"<h2 style='color:green'>Market Regime: {regime}</h2>"))

# ==============================
# 4️⃣ Indicator Calculations
# ==============================
def compute_indicators(df):
    df = df.copy()
    delta = df['Close'].diff()
    gain = delta.clip(lower=0).rolling(14).mean()
    loss = -delta.clip(upper=0).rolling(14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    df['SMA20'] = df['Close'].rolling(20).mean()
    df['SMA50'] = df['Close'].rolling(50).mean()
    df['ROC10'] = df['Close'].pct_change(10)
    df['VOLR'] = df['Volume'] / df['Volume'].rolling(20).mean()
    return df.dropna()

# ==============================
# 5️⃣ PyTorch LSTM Model
# ==============================
class MomentumLSTM(nn.Module):
    def __init__(self, input_size=5, hidden_size=32, num_layers=1, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        out = self.sigmoid(out)
        return out

# ==============================
# 6️⃣ Prepare sequences for LSTM
# ==============================
def prepare_sequences(df, seq_len=SEQ_LEN, future_days=FUTURE_DAYS):
    features = ['Close','Volume','RSI','ROC10','VOLR']
    df_clean = df[features].dropna()
    if len(df_clean) < seq_len + future_days:
        return None, None
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df_clean)
    X, y = [], []
    for i in range(len(scaled) - seq_len - future_days):
        X.append(scaled[i:i+seq_len])
        # Ensure each component is a scalar using .item()
        current_close = df_clean.iloc[i+seq_len]['Close'].item()
        future_close = df_clean.iloc[i+seq_len+future_days]['Close'].item()
        future_return = (future_close - current_close) / current_close
        y.append(1 if future_return > THRESHOLD else 0)
    return np.array(X), np.array(y)

# ==============================
# 7️⃣ Train LSTM & predict ai_prob
# ==============================
def train_and_predict(df):
    X, y = prepare_sequences(df)
    if X is None or len(X) < 20:
        return 0.5  # default if insufficient data
    split = int(len(X)*0.8)
    X_train, y_train = torch.FloatTensor(X[:split]).to(DEVICE), torch.FloatTensor(y[:split]).unsqueeze(1).to(DEVICE)
    X_test, y_test = torch.FloatTensor(X[split:]).to(DEVICE), torch.FloatTensor(y[split:]).unsqueeze(1).to(DEVICE)

    model = MomentumLSTM().to(DEVICE)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    model.train()
    for epoch in range(15):
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        latest_seq = torch.FloatTensor(X[-1:]).to(DEVICE)
        prob = model(latest_seq).item()
    return prob

# ==============================
# 8️⃣ Build Signals with AI Prob
# ==============================
rows = []
for t in TICKERS:
    df = yf.download(t, period="1y", progress=False, auto_adjust=True)
    df = compute_indicators(df)
    if len(df) < 60:
        continue
    # Get scalar values from the last row using .item() to ensure pure scalars
    last_close = df["Close"].iloc[-1].item()
    last_sma20 = df["SMA20"].iloc[-1].item()
    last_sma50 = df["SMA50"].iloc[-1].item()
    last_rsi = df["RSI"].iloc[-1].item()
    last_roc10 = df["ROC10"].iloc[-1].item()
    last_volr = df["VOLR"].iloc[-1].item()

    momentum_score = (
        int(last_close > last_sma20)*20 +
        int(last_sma20 > last_sma50)*20 +
        int(last_rsi > 55)*20 +
        int(last_roc10 > 0)*20 +
        int(last_volr > 1.3)*20
    )
    ai_prob = train_and_predict(df)  # 🔑 REAL LSTM prediction
    rows.append({
        "Ticker": t,
        "Price": round(last_close,2),
        "Momentum_Score": momentum_score,
        "AI_Probability": round(ai_prob,2),
        "Volume_Ratio": round(last_volr,2)
    })

signals = pd.DataFrame(rows).sort_values("AI_Probability", ascending=False)
display(HTML("<h3>Top AI Momentum Signals</h3>"))
display(signals.head(15))

# ==============================
# 9️⃣ Visualizations
# ==============================
# Confidence histogram
fig_conf = px.histogram(signals, x="AI_Probability", nbins=20, title="AI Momentum Confidence")
fig_conf.show()

# Equity curve simulation
equity = (1 + signals["AI_Probability"].values * 0.01).cumprod()
fig_eq = px.line(y=equity, title="Simulated Equity Curve (AI Weighted)")
fig_eq.show()


Using device: cpu


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1

/usr/local/lib/python3.12/dist-packages/

,Ticker,Price,Momentum_Score,AI_Probability,Volume_Ratio
17,MU,394.69,60,0.53,0.96
8,GOOGL,322.86,40,0.50,1.51
7,AMD,208.44,20,0.50,1.20
16,MRVL,80.28,40,0.49,1.51
5,TSLA,411.11,0,0.49,1.01
3,AMZN,210.32,40,0.49,3.47
12,PANW,159.32,20,0.49,1.37
4,META,661.46,80,0.49,0.95
2,NVDA,185.41,60,0.48,1.33
6,AVGO,332.92,40,0.48,1.18


In [ ]:
# ==============================
# 1️⃣ Install Packages
# ==============================
!pip install yfinance torch scikit-learn pandas numpy --quiet

import yfinance as yf
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
import os
from datetime import datetime, timedelta

# ==============================
# 2️⃣ Parameters
# ==============================
TICKERS = ["AAPL","MSFT","NVDA","AMZN","META"]
SEQ_LEN = 20
FUTURE_DAYS = 5
THRESHOLD = 0.02
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_PATH = "momentum_lstm.pth"
SCALER_PATH = "scaler.pkl"

# ==============================
# 3️⃣ LSTM Model Definition
# ==============================
class MomentumLSTM(nn.Module):
    def __init__(self, input_size=5, hidden_size=32, num_layers=1, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        out = self.sigmoid(out)
        return out

# ==============================
# 4️⃣ Prepare Sequences Function
# ==============================
def prepare_sequences(df, seq_len=SEQ_LEN, future_days=FUTURE_DAYS):
    features = ['Close','Volume','RSI','ROC10','VOLR']
    df_clean = df[features].dropna()
    if len(df_clean) < seq_len + future_days:
        return None, None, None # Also return None for scaler if data is insufficient
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df_clean)
    X, y = [], []
    for i in range(len(scaled) - seq_len - future_days):
        X.append(scaled[i:i+seq_len])
        # Ensure each component is a scalar using .item()
        current_close = df_clean.iloc[i+seq_len]['Close'].item()
        future_close = df_clean.iloc[i+seq_len+future_days]['Close'].item()
        future_return = (future_close - current_close) / current_close
        y.append(1 if future_return > THRESHOLD else 0)
    return np.array(X), np.array(y), scaler

# ==============================
# 5️⃣ Feature Engineering
# ==============================
def compute_indicators(df):
    df = df.copy()
    delta = df['Close'].diff()
    gain = delta.clip(lower=0).rolling(14).mean()
    loss = -delta.clip(upper=0).rolling(14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    df['SMA20'] = df['Close'].rolling(20).mean()
    df['SMA50'] = df['Close'].rolling(50).mean()
    df['ROC10'] = df['Close'].pct_change(10)
    df['VOLR'] = df['Volume'] / df['Volume'].rolling(20).mean()
    return df.dropna()

# ==============================
# 6️⃣ TRAIN LSTM (Run Once)
# ==============================
if not os.path.exists(MODEL_PATH):
    print("Training LSTM model from scratch...")
    all_X, all_y, scalers = [], [], {}
    for t in TICKERS:
        df = yf.download(t, period="3y", auto_adjust=True)
        df = compute_indicators(df)
        X, y, scaler = prepare_sequences(df)
        if X is not None:
            all_X.append(X)
            all_y.append(y)
            scalers[t] = scaler
    X_all = np.concatenate(all_X)
    y_all = np.concatenate(all_y)

    model = MomentumLSTM().to(DEVICE)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    X_tensor = torch.FloatTensor(X_all).to(DEVICE)
    y_tensor = torch.FloatTensor(y_all).unsqueeze(1).to(DEVICE)

    for epoch in range(30):
        optimizer.zero_grad()
        out = model(X_tensor)
        loss = criterion(out, y_tensor)
        loss.backward()
        optimizer.step()

    torch.save({
        'model_state_dict': model.state_dict(),
        'scalers': scalers
    }, MODEL_PATH)
    print("Model saved to", MODEL_PATH)
else:
    print("Model already trained, loading...")
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE, weights_only=False)
    model = MomentumLSTM().to(DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    scalers = checkpoint['scalers']
    model.eval()

# ==============================
# 7️⃣ DAILY INCREMENTAL PREDICTION
# ==============================
def daily_ai_prediction():
    results = []
    for t in TICKERS:
        df = yf.download(t, period="6mo", auto_adjust=True)  # small incremental fetch
        df = compute_indicators(df)
        if len(df) < SEQ_LEN + FUTURE_DAYS:
            print(f"Skipping {t}: Insufficient data after computing indicators. (Length: {len(df)} needs at least {SEQ_LEN + FUTURE_DAYS})")
            continue
        latest_features = ['Close','Volume','RSI','ROC10','VOLR']

        # Check if scaler exists for the current ticker
        if t not in scalers:
            print(f"Skipping {t}: Scaler not found. Ensure it was trained.")
            continue

        scaler = scalers[t]
        scaled = scaler.transform(df[latest_features])
        seq = torch.FloatTensor(scaled[-SEQ_LEN:]).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            prob = model(seq).item()

        last = df.iloc[-1]
        # Get scalar values from the last row using .item() to ensure pure scalars
        last_close = last['Close'].item()
        last_sma20 = last['SMA20'].item()
        last_sma50 = last['SMA50'].item()
        last_rsi = last['RSI'].item()
        last_roc10 = last['ROC10'].item()
        last_volr = last['VOLR'].item()

        momentum_score = (
            int(last_close > last_sma20)*20 +
            int(last_sma20 > last_sma50)*20 +
            int(last_rsi > 55)*20 +
            int(last_roc10 > 0)*20 +
            int(last_volr > 1.3)*20
        )
        results.append({
            'Ticker': t,
            'Price': round(last_close,2),
            'AI_Probability': round(prob,2),
            'Momentum_Score': momentum_score
        })

    if not results:
        print("No signals were generated for any ticker. Returning an empty DataFrame.")
        # Return an empty DataFrame with the expected columns to prevent KeyError
        return pd.DataFrame(columns=['Ticker', 'Price', 'AI_Probability', 'Momentum_Score'])

    return pd.DataFrame(results).sort_values("AI_Probability", ascending=False)

Model already trained, loading...


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1



In [ ]:
# ==============================
# 8️⃣ RUN DAILY PREDICTION
# ==============================
signals_today = daily_ai_prediction()
display(signals_today)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Ticker,Price,AI_Probability,Momentum_Score
1,MSFT,401.14,0.43,0
2,NVDA,185.41,0.42,60
3,AMZN,210.32,0.41,40
4,META,661.46,0.41,80
0,AAPL,278.12,0.40,60
